*please execute the cell below before starting the tutorial by selecting the cell and pressing Ctrl+Enter*

In [ ]:
from aiida.orm import Dict, load_node
from aiida.engine import submit

from aiida import load_profile

load_profile()

# Workchains and high-throughput study

In the final, sixth part of the AiiDA-FLEUR tutorial, we will cover the main part of AiiDA - workchains. Workchains are key-turn solutions for particular tasks such as finding equation of states or performing structure optimisation. The main goal of a workchain is to automatically perform a certain task keeping the history of all performed actions. The main advantage of using workchains is providing a low entering threshold for newcomers and routine tasks automatization for more advanced users.

In this part of the tutorial you will explore and learn how to work with AiiDA-Fleur workchains.

Workchains are similar to `CalcJob`: basically they can be treated as a black box using user's input and producing the output. For example, the SCF workchain is an engine that produces self-consistent charge density (and other interesting parameters) for a given structure. Equation Of States (EOS) workchain calculates equation of states for a given structure and etc.

<img src="files/images/black_box.png" width="1000">

The power of workchains is hidden behind their possible use in other workchains. Workchain A can be used in a workchain B, workchain C can use workchains A and B etc. Using smaller workchains as building blocks for a more complex algorithms, one can develop hierarchic structure of the task that AiiDA can perform.

## Workchain hierarchy 

The hierarchy of all implemented workchains in AiiDA-Fleur v1.0.0 is shown below. Black arrows mean the use of a workchain by another one. For instance, the geometry optimization workchain uses the SCF workchain inside it.
It is clearly seen that every higher-level workchain uses the SCF workchain.

<img src="files/images/workchains.png" width="600">


In next section we will cover the central workchain of the AiiDA-FLEUR plugin: the SCF workchain.

## SCF WorkChain 

The SCF workchain is responsible for converging charge density for a given structure. It submits a Fleur calculation several times until the convergence criterion or maximal number of Fleur submissions are reached.

### Inputs of the SCF WorkChain

|name|type|description| required |
|:---:|:---:|:---------:|:---:|
|fleur | Code | Fleur code | yes |
|inpgen | Code | Inpgen code| no |
|wf_parameters | Dict | Settings of the workchain| no |
|structure | StructureData | Structure data node| no |
|calc_parameters | Dict | FLAPW parameters, used by inpgen| no |
|fleurinp | FleurinpData | FLEUR input files| no |
|remote_data | RemoteData | Remote folder of another calculation| no |
|options | Dict | AiiDA options (computational resources)| no |
|settings | Dict | special settings for Fleur calculation| no |

Similarly to the `FleurCalculation`, SCF workchain has only one required input. Again, one *must* provide one of the supported input configurations to ensure the predictable behaviour of the workchain:

* **fleur** + **fleurinp**
* **fleur** + **fleurinp** + **remote_data**
* **fleur** + **remote_data**
* **fleur** + **inpgen** + **structure**

In this tutorial we will cover **fleur** + **inpgen** + **structure** configuration only, which makes the workchain to submit a single inpgen calculation followed by several Fleur code submissions.

## SCF workchain submission

### Inputs preparation

As always, import the `FleurScfWorkChain` first:

In [ ]:
from aiida_fleur.workflows.scf import FleurScfWorkChain

Next, let us setup input parameters for the workchain that control its behaviour. A cell below sets the maximal number of Fleur submission to 3, density convergence criterion to 0.001, maximal number of iterations in a single `FleurCalculation` to 30.

In [ ]:
wf_para = Dict(dict={'fleur_runmax' : 3,           # maximal number of Fleur submissions
                     'density_converge' : 0.001,   # density convergence criterion
                     'mode' : 'density',           # density is converged
                     'itmax_per_run' : 30,         # number of scf iterations in each Fleur submission
                     'serial' : False})            # use mpi submission

We are going to use StructureData node created in the tutorial number 1. Let us use the stored structure:

In [ ]:
# you need to modify this - remind the PK of the silicon structure SI_PK
structure = load_node(SI_PK)

Despite **options** and **wf_parameters** are never required, we will set them up because we do not want to use default values.

In [ ]:
options = Dict(dict={'resources' : {"num_machines": 1, "num_mpiprocs_per_machine" : 2},
                     'withmpi' : True,
                     'max_wallclock_seconds' : 600})

calc_parameters = Dict(dict={
    'kpt': {
        'div1': 2,
        'div2' : 2,
        'div3' : 2
        }})

**Note**: inpgen calculation submitted by SCF workchain is always submitted with an option:
 
    {'resources' : {"num_machines": 1, "num_mpiprocs_per_machine" : 1}, 'withmpi' : False}
    
which means the resources specified in SCF input apply to `FleurCalculation` only.

Finally, we need to load Fleur and inpgen nodes:

In [ ]:
fleur_code = load_node(146)
inpgen_code = load_node(7)

### Job submission

In contrast to `FluerinpgenCalculation` and `FleurCalculation`, there is no need to assemble inputs in a single dictionary (but one can do it for convenience).

In [ ]:
SCF_workchain = submit(FleurScfWorkChain,
                          fleur=fleur_code,
                          inpgen=inpgen_code,
                          calc_parameters=calc_parameters,
                          structure=structure,
                          wf_parameters=wf_para,
                          options=options)
print('Submitted SCF workchain pk={}'.format(SCF_workchain.pk))

Now we can check the status of the workchain simply executing a cell below:

In [ ]:
# you need to modify this
!verdi process status PK

You can execute the cell above again and again until it does not say the task is finished. The output for the finished workchain should look like:

<img src="files/images/scf_print.png" width="400">


You can also check all processes submitted last 24 hours by:

In [ ]:
!verdi process list -a -p 1

### Results analysis

SCF workchain returns three outputs:

|name| type | comment|
|:--:|:----:|:----------:|
|output_scf_wc_para|Dict| results of the workchain|
|fleurinp| FleurinpData| FleurinpData that was used (after all modifications) |
|last_fleur_calc_output| Dict | shortcut for last `FluerCalculation` output dict |

You can find all output node PKs running:

In [ ]:
# you need to modify this - replace SCF_PK
!verdi process show SCF_PK

Some data, given in `output_scf_wc_para` can be visualised via:

In [ ]:
# you need to modify this - replace SCF_PK
%matplotlib inline
from aiida_fleur.tools.plot.fleur import plot_fleur
plot_fleur(SCF_PK)

Finally, you can access output_scf_wc_para dictionary via:

In [ ]:
# you need to modify this - replace SCF_PK
scf_wc = load_node(SCF_PK)
scf_wc.outputs.output_scf_wc_para.get_dict()

or

In [ ]:
# you need to modify this - replace OUTPUT_DICT_PK
!verdi data dict show OUTPUT_DICT_PK

## Higher-level workchains

And now - finally - we will see the real power of the workchains. We are going to use two higher-level workchains for Magnetic Anisotropy Energy (MAE) and Spin Spiral Dispersion (SSDisp) calculations. There are two types of MAE and SSDisp workchains: force-theorem and convergence workchains. In this tutorial we will use force-theorem workchains only that first submits a single SCF workchain to obtain the reference charge density and later submits a single `FleurCalculation` task to run the force theorem step.

### Magnetic anisotropy workchain 

Import the workchain first:

In [ ]:
from aiida_fleur.workflows.mae import FleurMaeWorkChain

A table of possible inputs looks the same as for SCF:

|name|type|description| required |
|:---:|:---:|:---------:|:---:|
|fleur | Code | Fleur code | yes |
|inpgen | Code | Inpgen code| no |
|wf_parameters | Dict | Settings of the workchain| no |
|structure | StructureData | Structure data node| no |
|calc_parameters | Dict | FLAPW parameters, used by inpgen| no |
|fleurinp | FleurinpData | FLEUR input files| no |
|remote_data | RemoteData | Remote folder to find cdn1| no |
|options | Dict | AiiDA options (computational resources)| no |

Again, there are a lot of optional inputs - however you *must* follow one of the supported input configurations. They are similar to the SCF workchain and I will not cover all of them here, let us just use the **fleur** + **inpgen** + **structure** mode. Let us also specify workchain parameters and computational resources.

In [ ]:
options = Dict(dict={'resources' : {"num_machines": 1, "num_mpiprocs_per_machine" : 2},
                     'withmpi' : True,
                     'max_wallclock_seconds' : 600})

In [ ]:
wf_para = Dict(dict={'sqa_ref': [0.7, 0.7],                    # theta and phi for reference calculation
                     'use_soc_ref': False,                     # True if switch on SOC terms for reference
                     'sqas_theta': [0.0, 1.57079, 1.57079],    # a list of theta values to calculate via the FT
                     'sqas_phi': [0.0, 0.0, 1.57079],          # a list of phi values to calculate via the FT
                     'fleur_runmax': 10,                       # passed to SCF workchain
                     'density_converged': 0.02,                # passed to SCF workchain
                     'serial': False,                          # passed to SCF workchain
                     'itmax_per_run': 30,                       # passed to SCF workchain
                     'inpxml_changes': []                      # a list of inpxml changes to be done before submission
                    })

In [ ]:
calc_parameters = Dict(dict={
    'kpt': {
        'div1': 4,
        'div2' : 4,
        'div3' : 1
        }})

In [ ]:
fleur_code = load_node(???)
inpgen_code = load_node(???)

In this section we want to be even more productive - let us define not a single structure but three of them! Import Fe, Co and Ni structures that we created in tutorial 4:

In [ ]:
fe_structure = load_node(xxx)
co_structure = load_node(xxx)
ni_structure = load_node(xxx)

input_structures = [fe_structure, co_structure, ni_structure]

And calculate magnetic anisotropy energy for all of them:

In [ ]:
for structure in input_structures:
    MAE_workchain = submit(FleurMaeWorkChain,
                          fleur=fleur_code,
                          inpgen=inpgen_code,
                          calc_parameters=calc_parameters,
                          structure=structure,
                          wf_parameters=wf_para)
    print('Submitted Mae workchain pk={} for {} structure'.format(MAE_workchain.pk, structure.get_formula()))

Again, you can check if it is finished via:

In [ ]:
# you need to modify this - replace MAE_PK
!verdi process status MAE_PK

or

In [ ]:
!verdi process list -a -p 1

Using you experience, can you access the output dictionaries and extract values of MAEs for all the structures?

Now let us proceed to the final task - Spin spiral dispersion workchain.

# Scripting tasks

### Spin-spiral dispersion workchain

Spin spiral dispersion workchain has the same input nodes as MAE workchain. `wf_para` dictionary contains a different set of control keys:

In [ ]:
wf_para = Dict(dict={'fleur_runmax' : 3,                    # passed to SCF workchain
                     'itmax_per_run' : 30,                  # passed to SCF workchain
                     'density_converged' : 0.002,           # passed to SCF workchain
                     'serial' : False,                      # passed to SCF workchain
                     'beta' : {'all' : 1.57079},            # sets beta angle for all atoms
                     'q_vectors': [[0.0, 0.0, 0.0,          # set q-vectors to calculate
                                   [0.125, 0.125, 0.0],     
                                   [0.250, 0.250, 0.0],     
                                   [0.375, 0.375, 0.0],     
                                   [0.500, 0.500, 0.0]],     
                     'ref_qss' : [0.0, 0.0, 0.0],            # sets a q-vector of the reference calc
                     'inpxml_changes': []
                    })

To import SSDisp workchain, run:

    from aiida_fleur.workflows.ssdisp import FleurSSDispWorkChain

 This time we will cover another input configuration: `fleur + fleurinp`. We will use `FleurinpData` objects generated in section 4.
 
In the final task, load three `FleurinpData` objects and run a `FleurSSDispWorkChain` for each of them. Use **wf_para** given above. Do you need to pass **calc_parameters** this time?

Explore the outputs, extract spin spiral dispersion. Plot energy of a spin spiral as a function of a q-vector.